In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy.spatial.distance import cdist
from skimage import measure
from matplotlib import pyplot as plt
import os
import tifffile
from tqdm import tqdm
from typing import List, Tuple

In [2]:
selected_magnification = "20x"
selected_image_type = "dw"
dw__thr = 13
max_dist = 9
aspect = np.array((1/3, 1/3))    # XY: 20x / aspect => 60x

### Read shift data

In [3]:
shifts = pd.read_csv(f"shift_correction/{selected_magnification}_{selected_image_type}.shifts.csv")
shifts.index = shifts["sid"].values
shifts.drop("sid", 1, inplace=True)

# Matching 20x_dw and reference dots

In [4]:
dots_data = pd.read_csv("/mnt/data/Imaging/202105-Deconwolf/data_210726/dots_data.clean.tsv.gz", sep="\t")
dots_data = dots_data[selected_magnification == dots_data["magnification"]]
dots_data = dots_data[selected_image_type == dots_data["image_type"]]

In [5]:
reference = pd.read_csv("../../data/60x_reference/ref__dw.global_thr.tsv", sep="\t")

In [6]:
def get_unassigned_count(pdist: np.ndarray) -> int:
    return np.sum(np.nansum(np.isnan(pdist), 1) != pdist.shape[1])

def match_dots(dotsA: np.ndarray, dotsB: np.ndarray) -> List[Tuple[int, int, float]]:
    pdist = cdist(dotsA, dotsB)
    
    matched: List[Tuple[int, int, float]] = []
    unassigned_rows = pdist.shape[0] + 1
    iteration_counter = 0
    print("Iteration #", end="")
    while unassigned_rows > get_unassigned_count(pdist) and 0 < get_unassigned_count(pdist):
        iteration_counter += 1
        print(f"...{iteration_counter}", end="")

        unassigned_rows = get_unassigned_count(pdist)
        A_id = 0
        while A_id < pdist.shape[0]:
            if pdist.shape[1] == np.sum(np.isnan(pdist[A_id, :])):
                A_id += 1
                continue
            B_id = np.nanargmin(pdist[A_id, :])
            if np.nanargmin(pdist[:, B_id]) == A_id:
                matched.append((A_id, B_id, pdist[A_id, B_id]))
                pdist[A_id, :] = np.nan
                pdist[:, B_id] = np.nan
            A_id += 1
    print("")
    return matched

In [7]:
match_output: List[pd.DataFrame] = []
match_output_full: List[pd.DataFrame] = []
match_counts: List[Tuple[int, int, int, int]] = []
for current_field_id in tqdm(np.unique(dots_data["sid"])):    
    dots = dots_data.loc[current_field_id == dots_data["sid"], :].copy(
        ).sort_values("Value2", ascending=False).reset_index(drop=True)
    
    dot_coords = dots.loc[dw__thr <= dots["Value2"], ("x", "y")].copy().reset_index(drop=True)
    dot_coords2 = dot_coords.copy() / aspect
    dot_coords2["x"] += (shifts.loc[current_field_id, "x"] * 9)
    dot_coords2["y"] += (shifts.loc[current_field_id, "y"] * 9)
    ref_coords = reference.loc[reference["sid"] == current_field_id, ("x", "y")].copy().reset_index(drop=True)
    
    matched: List[Tuple[int, int, float]] = match_dots(dot_coords2, ref_coords)
    
    matched_a = np.array(matched)
    matched_field = pd.DataFrame(dict(
        series=current_field_id,
        id_20x=matched_a[:, 0].astype("i"),
        id_60x=matched_a[:, 1].astype("i"),
        eudist=matched_a[:, 2]
    ))
    matched_field_within_dist = matched_field[matched_field["eudist"] <= max_dist]
    match_output.append(matched_field_within_dist)
    match_output_full.append(matched_field)
    match_counts.append((current_field_id,
                         matched_field_within_dist.shape[0],
                         matched_field.shape[0]-matched_field_within_dist.shape[0],
                         dot_coords2.shape[0],
                         ref_coords.shape[0]))

  0%|          | 0/7 [00:00<?, ?it/s]

Iteration #...1...2...3...4

 14%|█▍        | 1/7 [00:00<00:02,  2.40it/s]

...5...6...7...8
Iteration #...1...2...3...4...5...6...7...8...9...10

 29%|██▊       | 2/7 [00:01<00:02,  1.88it/s]

...

...9
Iteration #...1...2...3...4...5...6...7...8

 71%|███████▏  | 5/7 [00:03<00:01,  1.53it/s]

...9...10
Iteration #...1...2...3...4...5...6...7...8...9...10...11...12...13...14...15...16...17


100%|██████████| 7/7 [00:03<00:00,  1.92it/s]

Iteration #...1...2...3...4...5...6...7...8...9...10...11...12...13...14...15...16...17...18...19...20


In [8]:
match_counts_a = np.array(match_counts)
n_total_matched = np.sum(match_counts_a[:, 1])
n_total_over_dist = np.sum(match_counts_a[:, 2])
n_total_20x = np.sum(match_counts_a[:, 3])
n_total_60x = np.sum(match_counts_a[:, 4])
print(f"{n_total_matched}/{n_total_20x} ({n_total_matched/n_total_20x*100:.2f}%) 20x dots matched to 60x")
print(f"{n_total_matched}/{n_total_60x} ({n_total_matched/n_total_60x*100:.2f}%) 60x dots matched to 20x")
print(f"{n_total_over_dist} ({n_total_over_dist/n_total_20x*100:.2f}% 20x, {n_total_over_dist/n_total_60x*100:.2f}% 60x) dots matched with distance > {max_dist}")

6595/7948 (82.98%) 20x dots matched to 60x
6595/9440 (69.86%) 60x dots matched to 20x
1353 (17.02% 20x, 14.33% 60x) dots matched with distance > 9


In [9]:
for series_id in range(match_counts_a.shape[0]):
    n_matched = match_counts_a[series_id, 1]
    n_over_dist = match_counts_a[series_id, 2]
    n_20x = match_counts_a[series_id, 3]
    n_60x = match_counts_a[series_id, 4]
    print(f"""Field #{series_id+1}
    {n_matched}/{n_20x} ({n_matched/n_20x*100:.2f}%) 20x dots matched to 60x
    {n_matched}/{n_60x} ({n_matched/n_60x*100:.2f}%) 60x dots matched to 20x
    {n_over_dist} ({n_over_dist/n_20x*100:.2f}% 20x, {n_over_dist/n_60x*100:.2f}% 60x) dots matched with distance > {max_dist}
    """)

Field #1
    994/1191 (83.46%) 20x dots matched to 60x
    994/1308 (75.99%) 60x dots matched to 20x
    197 (16.54% 20x, 15.06% 60x) dots matched with distance > 9
    
Field #2
    1423/1696 (83.90%) 20x dots matched to 60x
    1423/1921 (74.08%) 60x dots matched to 20x
    273 (16.10% 20x, 14.21% 60x) dots matched with distance > 9
    
Field #3
    745/879 (84.76%) 20x dots matched to 60x
    745/1045 (71.29%) 60x dots matched to 20x
    134 (15.24% 20x, 12.82% 60x) dots matched with distance > 9
    
Field #4
    1176/1364 (86.22%) 20x dots matched to 60x
    1176/1548 (75.97%) 60x dots matched to 20x
    188 (13.78% 20x, 12.14% 60x) dots matched with distance > 9
    
Field #5
    1691/2003 (84.42%) 20x dots matched to 60x
    1691/2404 (70.34%) 60x dots matched to 20x
    312 (15.58% 20x, 12.98% 60x) dots matched with distance > 9
    
Field #6
    415/557 (74.51%) 20x dots matched to 60x
    415/691 (60.06%) 60x dots matched to 20x
    142 (25.49% 20x, 20.55% 60x) dots matched 

#### General statistics, excluding field #7

In [10]:
match_counts_a = np.array(match_counts)
n_total_matched = np.sum(match_counts_a[:6, 1])
n_total_over_dist = np.sum(match_counts_a[:6, 2])
n_total_20x = np.sum(match_counts_a[:6, 3])
n_total_60x = np.sum(match_counts_a[:6, 4])
print(f"{n_total_matched}/{n_total_20x} ({n_total_matched/n_total_20x*100:.2f}%) 20x dots matched to 60x")
print(f"{n_total_matched}/{n_total_60x} ({n_total_matched/n_total_60x*100:.2f}%) 60x dots matched to 20x")
print(f"{n_total_over_dist} ({n_total_over_dist/n_total_20x*100:.2f}% 20x, {n_total_over_dist/n_total_60x*100:.2f}% 60x) dots matched with distance > {max_dist}")

6444/7690 (83.80%) 20x dots matched to 60x
6444/8917 (72.27%) 60x dots matched to 20x
1246 (16.20% 20x, 13.97% 60x) dots matched with distance > 9


#### General statistics, excluding fields #6-7

In [11]:
match_counts_a = np.array(match_counts)
n_total_matched = np.sum(match_counts_a[:5, 1])
n_total_over_dist = np.sum(match_counts_a[:5, 2])
n_total_20x = np.sum(match_counts_a[:5, 3])
n_total_60x = np.sum(match_counts_a[:5, 4])
print(f"{n_total_matched}/{n_total_20x} ({n_total_matched/n_total_20x*100:.2f}%) 20x dots matched to 60x")
print(f"{n_total_matched}/{n_total_60x} ({n_total_matched/n_total_60x*100:.2f}%) 60x dots matched to 20x")
print(f"{n_total_over_dist} ({n_total_over_dist/n_total_20x*100:.2f}% 20x, {n_total_over_dist/n_total_60x*100:.2f}% 60x) dots matched with distance > {max_dist}")

6029/7133 (84.52%) 20x dots matched to 60x
6029/8226 (73.29%) 60x dots matched to 20x
1104 (15.48% 20x, 13.42% 60x) dots matched with distance > 9


### Export

In [12]:
matched_dots = pd.concat(match_output)

In [13]:
matched_dots.to_csv(os.path.join("../../data/magnifications_matching",
                                 f"{selected_magnification}_{selected_image_type}.matched_dots.global_thr.tsv"
                                ), sep="\t", index=False)

In [14]:
pd.DataFrame(match_counts_a, columns=["series_id", "matched", "matched_over_dist", "raw_total", "dw_total"]
            ).to_csv(os.path.join("../../data/magnifications_matching",
                                  f"{selected_magnification}_{selected_image_type}.matched_dots.global_thr.counts.tsv"
                                 ), sep="\t", index=False)